In [1]:
'''
Imports
'''
import IPython.display as ipd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as skl
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm
import librosa
import librosa.display
import ast
import os

# Data loading and preprocessing

## As first thing, we load the csv files

In [8]:
def get_mfcc(y, sr):
    mfcc = np.array(librosa.feature.mfcc(y=y, sr=sr))
    return mfcc

def get_melspectrogram(y, sr):
    melspectrogram = np.array(librosa.feature.melspectrogram(y=y, sr=sr))
    return melspectrogram

def get_chroma_stft(y, sr):
    chroma_stft = np.array(librosa.feature.chroma_stft(y=y, sr=sr))
    return chroma_stft

def get_tonnetz(y, sr):
    tonnetz = np.array(librosa.feature.tonnetz(y=y, sr=sr))
    return tonnetz

def get_chroma_cqt(y, sr):
    chroma_cqt = np.array(librosa.feature.chroma_cqt(y=y, sr=sr))
    return chroma_cqt

def get_chroma_cens(y, sr):
    chroma_cens = np.array(librosa.feature.chroma_cens(y=y, sr=sr))
    return chroma_cens

def get_spectral_centroid(y, sr):
    spectral_centroid = np.array(librosa.feature.spectral_centroid(y=y, sr=sr))
    return spectral_centroid

def get_spectral_bandwidth(y, sr):
    spectral_bandwidth = np.array(librosa.feature.spectral_bandwidth(y=y, sr=sr))
    return spectral_bandwidth

def get_spectral_contrast(y, sr):
    spectral_contrast = np.array(librosa.feature.spectral_contrast(y=y, sr=sr))
    return spectral_contrast

def get_spectral_flatness(y, sr):
    spectral_flatness = np.array(librosa.feature.spectral_flatness(y=y, sr=sr))
    return spectral_flatness

def get_spectral_rolloff(y, sr):
    spectral_rolloff = np.array(librosa.feature.spectral_rolloff(y=y, sr=sr))
    return spectral_rolloff

def get_poly_features(y, sr):
    poly_features = np.array(librosa.feature.poly_features(y=y, sr=sr))
    return poly_features

def get_rms(y, sr):
    rms = np.array(librosa.feature.rms(y=y, sr=sr))
    return rms

def get_zero_crossing_rate(y, sr):
    zero_crossing_rate = np.array(librosa.feature.zero_crossing_rate(y=y, sr=sr))
    return zero_crossing_rate

def get_tempogram(y, sr):
    tempogram = np.array(librosa.feature.tempogram(y=y, sr=sr))
    return tempogram

def get_fourier_tempogram(y, sr):
    fourier_tempogram = np.array(librosa.feature.fourier_tempogram(y=y, sr=sr))
    return fourier_tempogram

file_path = "C:\\Users\\Gian Marco\\Desktop\\Music-Learning\\dataset\\fma_medium\\000\\000002.mp3"
x, sr = librosa.load(file_path, sr=None, mono=True)
#y, sr = librosa.load(file_path, offset=0, duration=30)
"""get_chroma_stft(y, sr)
get_chroma_cens(y, sr)
get_chroma_cqt(y, sr)
get_fourier_tempogram(y, sr)
get_melspectrogram(y, sr)
get_mfcc(y, sr)
get_poly_features(y, sr)
get_rms(y, sr)
get_spectral_bandwidth(y, sr)
get_spectral_centroid(y, sr)
get_spectral_contrast(y, sr)
get_spectral_flatness(y, sr)
get_spectral_rolloff(y, sr)
get_tempogram(y, sr)
get_tonnetz(y, sr)
get_zero_crossing_rate(y, sr)"""

NoBackendError: 

In [ ]:
def get_feature(file_path):
    # Extracting MFCC feature
    mfcc = get_mfcc(file_path)
    mfcc_mean = mfcc.mean(axis=1)
    mfcc_min = mfcc.min(axis=1)
    mfcc_max = mfcc.max(axis=1)
    mfcc_feature = np.concatenate( (mfcc_mean, mfcc_min, mfcc_max) )

    # Extracting Mel Spectrogram feature
    melspectrogram = get_melspectrogram(file_path)
    melspectrogram_mean = melspectrogram.mean(axis=1)
    melspectrogram_min = melspectrogram.min(axis=1)
    melspectrogram_max = melspectrogram.max(axis=1)
    melspectrogram_feature = np.concatenate( (melspectrogram_mean, melspectrogram_min, melspectrogram_max) )

    # Extracting chroma vector feature
    chroma = get_chroma_stft(file_path)
    chroma_mean = chroma.mean(axis=1)
    chroma_min = chroma.min(axis=1)
    chroma_max = chroma.max(axis=1)
    chroma_feature = np.concatenate( (chroma_mean, chroma_min, chroma_max) )

    # Extracting tonnetz feature
    tntz = get_tonnetz(file_path)
    tntz_mean = tntz.mean(axis=1)
    tntz_min = tntz.min(axis=1)
    tntz_max = tntz.max(axis=1)
    tntz_feature = np.concatenate( (tntz_mean, tntz_min, tntz_max) ) 
    
    feature = np.concatenate( (chroma_feature, melspectrogram_feature, mfcc_feature, tntz_feature) )
    return feature